<a href="https://colab.research.google.com/github/sanaz-mahmoudi/sanazmahmoudi/blob/main/Newcastle_University_Interview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pre Interview Task**

## Requirements

In [15]:
# PYOMO
!pip install -q pyomo
import pyomo.environ as pe

# NUMPY
import numpy as np

# GLPK
!apt-get install -y -qq glpk-utils
glpk = pe.SolverFactory('glpk', executable='/usr/bin/glpsol')

# PyPlot
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# Set font configuration at the start
!wget -O TimesNewRoman.ttf \
  https://github.com/justrajdeep/fonts/raw/master/Times%20New%20Roman.ttf

import matplotlib.font_manager as fm
import matplotlib as mpl

fm.fontManager.addfont('TimesNewRoman.ttf')

mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = ['Times New Roman']

# OTHER
import pandas as pd

--2025-08-02 09:59:54--  https://github.com/justrajdeep/fonts/raw/master/Times%20New%20Roman.ttf
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/justrajdeep/fonts/master/Times%20New%20Roman.ttf [following]
--2025-08-02 09:59:55--  https://raw.githubusercontent.com/justrajdeep/fonts/master/Times%20New%20Roman.ttf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 834452 (815K) [application/octet-stream]
Saving to: ‘TimesNewRoman.ttf’

TimesNewRoman.ttf   100%[===================>] 814.89K  --.-KB/s    in 0.06s   

2025-08-02 09:59:55 (13.3 MB/s) - ‘TimesNewRoman.ttf’ saved [834452/834452]



## Input Data

In [16]:
print('=====================================')
print('           Load Demand Data          ')
print('=====================================')
data_dem = [[0],[0],[100]]
nn = len(data_dem)
dem = pd.DataFrame(data_dem,index=['n'+str(i) for i in range(nn)],columns=['pmax'])
print(dem)

print('=====================================')
print('              Line Data              ')
print('=====================================')
data_lin = [[0,1,100],[1,2,100],[0,2,100]]
lin = pd.DataFrame(np.zeros((nn, nn), dtype=int), index=range(nn), columns=range(nn))
for from_bus, to_bus, pmax in data_lin:
    lin.loc[from_bus, to_bus] = pmax
    lin.loc[to_bus, from_bus] = pmax
print(lin)

print('=====================================')
print('       Thermal Power Plant Data      ')
print('=====================================')
data_gen = [[0,0,250,50]]
ng = len(data_gen)
gen = pd.DataFrame(data_gen,index=['g'+str(i) for i in range(ng)],columns=['bus','pmin','pmax','o_cost'])
print(gen)

print('=====================================')
print('      Sulphur-Flow Battery Data      ')
print('=====================================')
data_bat = [[1,100,0,2400,0,100,0,100,0.9,0.9]]
nb = len(data_bat)
bat = pd.DataFrame(data_bat,index=['b'+str(i) for i in range(nb)],
                   columns=['bus','eini','emin','emax','pcmin','pcmax','pdmin','pdmax','etac','etad'])
print(bat)

print('=====================================')
print('            Renewable Data           ')
print('=====================================')
data_ren = [[1,0,150,0]]
nw = len(data_ren)
ren = pd.DataFrame(data_ren,index=['w'+str(i) for i in range(nw)],columns=['bus','pmin','pmax','o_cost'])
print(ren)

print('=====================================')
print(' Hourly Demand and Renewable Profile ')
print('=====================================')
h_data = pd.read_csv('https://raw.githubusercontent.com/sanaz-mahmoudi/sanazmahmoudi/refs/heads/main/Hourly%20Profiles.csv')

nd = 28;  # Number of Days
nh = 24;  # Number of Hours

d_profile = h_data['Normalised Electricity Consumption Profile'].values.reshape((nd, nh))
pd = np.zeros((nd, nh, nn))
for i in range(nn):
    pd[:, :, i] = d_profile * dem['pmax'].values[i]
print('Hourly Load Demand =\n', pd)

r_profile = h_data['Normalised Electricity Consumption Profile'].values.reshape((nd, nh))
pr = np.zeros((nd, nh, nw))
for i in range(nw):
    pr[:, :, i] = r_profile * ren['pmax'].values[i]
print('Hourly Renewable Generstion =\n', pr)

print('=====================================')
print('     PyPlots for Hourly Profiles     ')
print('=====================================')
# Reshape into Groups of 7*24 (days*hours) Rows per Column
re_h_ren_data = h_data['Normalised Renewable Production Profile'].values.reshape(-1, 7*24).T
re_h_dem_data = h_data['Normalised Electricity Consumption Profile'].values.reshape(-1, 7*24).T

# Convert to DataFrame and Rename Columns
w_ren_data = pd.DataFrame(re_h_ren_data, columns=[f'week{i+1}' for i in range(re_h_ren_data.shape[1])])
print(w_ren_data)
w_dem_data = pd.DataFrame(re_h_dem_data, columns=[f'week{i+1}' for i in range(re_h_dem_data.shape[1])])
print(w_dem_data)

# -------------------- Plot 1: Renewable Production --------------------
colors = plt.cm.get_cmap('winter', w_ren_data.shape[1])

plt.figure(figsize=(10, 6))

x = np.arange(len(w_ren_data.index))

for i, week in enumerate(w_ren_data.columns):
    plt.plot(x, w_ren_data[week] * ren['pmax'].values, label=week, color=colors(i))

# Legend
legend_lines = [Line2D([0], [0], color=colors(i), lw=2) for i in range(w_ren_data.shape[1])]
legend_labels = [f'Week {i+1}' for i in range(w_ren_data.shape[1])]
plt.legend(legend_lines, legend_labels, loc='upper left', bbox_to_anchor=(1.02, 1), fontsize=14)

# Axes
tick_positions = [1] + [24 * i for i in range(1, 8)]
tick_positions = [t for t in tick_positions if t <= 168]
plt.xticks(tick_positions, [str(t) for t in tick_positions], fontsize=12)
plt.yticks(fontsize=12)

# Labels
plt.xlabel('Hours (h)', fontname='Times New Roman', fontsize=18)
plt.ylabel('Renewable Production (MW)', fontname='Times New Roman', fontsize=18)
plt.title('Renewable Production Profile', fontname='Times New Roman', fontsize=18)
plt.tight_layout()
plt.show()

# -------------------- Plot 2: Electricity Consumption --------------------
# REDEFINE colormap, lines, labels for the second figure
colors = plt.cm.get_cmap('winter', w_dem_data.shape[1])

plt.figure(figsize=(10, 6))

x = np.arange(len(w_dem_data.index))

for i, week in enumerate(w_dem_data.columns):
    plt.plot(x, w_dem_data[week] * dem['pmax'].values, label=f'Week {i+1}', color=colors(i))

# Legend
legend_lines = [Line2D([0], [0], color=colors(i), lw=2) for i in range(w_dem_data.shape[1])]
legend_labels = [f'Week {i+1}' for i in range(w_dem_data.shape[1])]
plt.legend(legend_lines, legend_labels, loc='upper left', bbox_to_anchor=(1.02, 1), fontsize=14)

# Axes
tick_positions = [1] + [24 * i for i in range(1, 8)]
tick_positions = [t for t in tick_positions if t <= 168]
plt.xticks(tick_positions, [str(t) for t in tick_positions], fontsize=12)
plt.yticks(fontsize=12)

# Labels
plt.xlabel('Hours (h)', fontname='Times New Roman', fontsize=18)
plt.ylabel('Electricity Consumption (MW)', fontname='Times New Roman', fontsize=18)
plt.title('Electricity Consumption Profile', fontname='Times New Roman', fontsize=18)
plt.tight_layout()
plt.show()


           Load Demand Data          
    pmax
n0     0
n1     0
n2   100
              Line Data              
     0    1    2
0    0  100  100
1  100    0  100
2  100  100    0
       Thermal Power Plant Data      
    bus  pmin  pmax  o_cost
g0    0     0   250      50
      Sulphur-Flow Battery Data      
    bus  eini  emin  emax  pcmin  pcmax  pdmin  pdmax  etac  etad
b0    1   100     0  2400      0    100      0    100   0.9   0.9
            Renewable Data           
    bus  pmin  pmax  o_cost
w0    1     0   150       0
 Hourly Demand and Renewable Profile 
Hourly Load Demand =
 [[[ 0.         0.        19.1994838]
  [ 0.         0.        15.5534504]
  [ 0.         0.        10.650177 ]
  ...
  [ 0.         0.        41.0018387]
  [ 0.         0.        28.783375 ]
  [ 0.         0.        15.4068781]]

 [[ 0.         0.        14.4110813]
  [ 0.         0.        12.8498808]
  [ 0.         0.        10.8773081]
  ...
  [ 0.         0.        43.0098424]
  [ 0.         0. 

AttributeError: 'numpy.ndarray' object has no attribute 'DataFrame'

## Optimization Model

In [ ]:
# Model
m = pe.ConcreteModel()

# Sets
m.b = pe.Set(initialize=list(range(nb)))                         # Set of Sulphur-Flow Batteries
m.d = pe.Set(initialize=list(range(nd)))                         # Set of Load Demand Centers
m.g = pe.Set(initialize=gen.index.tolist())                      # Set of Thermal Power Plants
m.l = pe.Set(initialize=list(range(nl)))                         # Set of Lines
m.n = pe.Set(initialize=list(range(nb)))                         # Set of Nodes
m.td = pe.Set(initialize=list(h_data['day'].unique()))           # Set of Days
m.th = pe.Set(initialize=list(h_data['hour'].unique()))          # Set of Hours
m.w = pe.Set(initialize=list(range(nw)))                         # Set of Renewable Sources

# Variables
m.bcb = pe.Var(m.b,m.td,m.th,within=pe.Binary)                   # Charging Status of Sulphur-Flow Batteries
m.bdb = pe.Var(m.b,m.td,m.th,within=pe.Binary)                   # Discharging Status of Sulphur-Flow Batteries
m.pcb = pe.Var(m.b,m.td,m.th,within=pe.NonNegativeReals)         # Power Charging of Sulphur-Flow Batteries
m.pdb = pe.Var(m.b,m.td,m.th,within=pe.NonNegativeReals)         # Power Discharging of Sulphur-Flow Batteries
m.pds = pe.Var(m.d,m.td,m.th,within=pe.Binary)                   # Demand Shedding
m.pg  = pe.Var(m.g,m.td,m.th,within=pe.NonNegativeReals)         # Power Production of Thermal Power Plants
m.pl  = pe.Var(m.l,m.td,m.th,within=pe.Reals)                    # Power Flow of Lines
m.prs = pe.Var(m.w,m.td,m.th,within=pe.NonNegativeReals)         # Renewable Shedding
m.pw  = pe.Var(m.w,m.td,m.th,within=pe.NonNegativeReals)         # Power Production of Renewable Sources

# Objective function
def obj_rule(m):
  return sum(gen['o_cost'][g]*m.pg[g,td,th] for g in m.g for td in m.td for th in m.th)
m.obj = pe.Objective(rule=obj_rule)

# Constraints
# Energy balance
def bal_rule(m,n,td,th):
  return sum(m.pg[g,td,th] for g in m.g if gen['bus'][g] == n) + \
         sum(m.pw[w,td,th] for w in m.w if ren['bus'][w] == n) + \
         sum(m.pdb[b,td,th] for b in m.b if bat['bus'][b] == n) + \
         sum(m.pl[l,td,th] for l in m.l if lin['to'][l] == n) == \
         sum(m.pl[l,td,th] for l in m.l if lin['from'][l] == n) + \
         sum(m.pcb[b,td,th] for b in m.b if bat['bus'][b] == n) + \
         sum(h_dem.iloc[d,td,th] for d in m.d if h_dem['bus'][d] == n)
m.bal = pe.Constraint(m.n, m.td, m.th, rule=bal_rule)

# ------------------- Thermal Power Plants --------------------
# Minimum Thermal Generation Capacity
def min_gen_rule(m,g,td,th):
    return m.pg[g,td,th] >= gen['pmin'][g]
m.min_gen = pe.Constraint(m.g, m.td, m.th, rule=min_gen_rule)

# Maximum Thermal Generation Capacity
def max_gen_rule(m,g,td,th):
    return m.pg[g,td,th] <= gen['pmax'][g]
m.max_gen = pe.Constraint(m.g, m.td,  m.th, rule=max_gen_rule)

# --------------------- Renewable Sources ---------------------
# Minimum Renewable Generation Capacity
def min_ren_rule(m,w,td,th):
    return m.pw[w,td,th] >= ren['pmin'][w]
m.min_ren = pe.Constraint(m.w, m.td, m.th, rule=min_ren_rule)

# Maximum Renewable Generation Capacity
def max_ren_rule(m,w,td,th):
    return m.pw[w,td,th] <= ren['pmax'][w]
m.max_ren = pe.Constraint(m.w, m.td, m.th, rule=max_ren_rule)

# ------------------- Sulphur-Flow Battery --------------------
# Minimum Charging Limit of Sulphur-Flow Battery
def min_cha_rule(m,b,td,th):
    return m.pcb[b,td,th] >= bat['pcmin'][b]*m.bcb[b,td,th]
m.min_cha = pe.Constraint(m.b, m.td, m.th, rule=min_cha_rule)

# Maximum Charging Limit of Sulphur-Flow Battery
def max_cha_rule(m,b,td,th):
    return m.pcb[b,td,th] <= bat['pcmax'][b]*m.bcb[b,td,th]
m.max_cha = pe.Constraint(m.b, m.td, m.th, rule=max_cha_rule)

# Minimum Discharging Limit of Sulphur-Flow Battery
def min_dis_rule(m,b,td,th):
    return m.pdb[b,td,th] >= bat['pdmin'][b]*m.bdb[b,td,th]
m.min_dis = pe.Constraint(m.b, m.td, m.th, rule=min_dis_rule)

# Maximum Discharging Limit of Sulphur-Flow Battery
def max_dis_rule(m,b,td,th):
    return m.pdb[b,td,th] <= bat['pdmax'][b]*m.bdb[b,td,th]
m.max_dis = pe.Constraint(m.b, m.td, m.th, rule=max_dis_rule)

# Minimum Stored Energy of Sulphur-Flow Battery
def min_sto_rule(m,b,td):
    return bat['emin'][b] <= bat['eini'][b] + \
    sum(bat['etac'][b]*m.pcb[b,td,th] - (1/bat['etad'][b])*m.pdb[b,td,th] for th in m.th)
m.min_sto = pe.Constraint(m.b, m.td, rule=min_sto_rule)

# Maximum Stored Energy of Sulphur-Flow Battery
def max_sto_rule(m,b,td):
    return bat['eini'][b] + sum(bat['etac'][b]*m.pcb[b,td,th] - \
     (1/bat['etad'][b])*m.pdb[b,td,th] for th in m.th) <= bat['emax'][b]
m.max_sto = pe.Constraint(m.b, m.td, rule=max_sto_rule)

# Binding Initial and Final Stored Energy Levels of Sulphur-Flow Battery
def bid_sto_rule(m,b,td):
    return sum(bat['etac'][b]*m.pcb[b,td,th] - (1/bat['etad'][b])*m.pdb[b,td,th] for th in m.th) == 0
m.bid_sto = pe.Constraint(m.b, m.td, rule=bid_sto_rule)

# No Simultaneously Charges and Discharges
def cha_dis_rule(m,b,td,th):
    return m.bcb[b,td,th] + m.bdb[b,td,th] <= 1
m.cha_dis = pe.Constraint(m.b, m.td, m.th, rule=cha_dis_rule)

# -------------------------- Lines ----------------------------
# Minimum Line Flow
def min_flow_rule(m,l,td,th):
    return m.pl[l,td,th] >= -lin['pmax'][l]
m.min_flow = pe.Constraint(m.l, m.td, m.th, rule=min_flow_rule)

# Maximum Line Flow
def max_flow_rule(m,l,td,th):
    return m.pl[l,td,th] <= lin['pmax'][l]
m.max_flow = pe.Constraint(m.l, m.td, m.th, rule=max_flow_rule)

# Solve problem using GLPK solver
glpk.solve(m).write()

# Print results
#print('x1 =',m.x1.value)
#print('x2 =',m.x2.value)
print('Optimal Value =',m.obj())